In [1]:
!pip install -q streamlit pyngrok transformers accelerate torch


In [2]:
NGROK_AUTHTOKEN = "31EF4pPCBK135AfzktlSmABX4IN_35kdCYYL1rMJy4b8ccN3x"  # paste your token here

if NGROK_AUTHTOKEN:
    from pyngrok import conf
    conf.get_default().auth_token = NGROK_AUTHTOKEN
    print("ngrok token set ✅")
else:
    print("No token set — if you see 401 errors later, rerun with token.")


ngrok token set ✅


In [9]:
%%writefile app.py
import os, json, requests, torch
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

st.set_page_config(page_title="💰 Personal Finance Chatbot", layout="wide")
st.title("💰 Personal Finance Chatbot")
st.caption("Intelligent guidance for savings, taxes, and investments.")

@st.cache_resource
def load_granite():
    tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = AutoModelForCausalLM.from_pretrained(
        "ibm-granite/granite-3.3-2b-instruct",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map=None
    ).to(device)
    return tokenizer, model, device

tokenizer, model, device = load_granite()

def generate_response(prompt, target_points=6, chunk_size=250):
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(device)

    output_text = ""
    current_points = 0

    while current_points < target_points:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=chunk_size,
                eos_token_id=None,
                pad_token_id=tokenizer.eos_token_id
            )

        new_text = tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )

        output_text += new_text
        current_points = output_text.count("\n")  # crude way: count list points
        inputs = tokenizer(output_text, return_tensors="pt").to(device)

        if current_points >= target_points:
            break

    return output_text.strip()

from transformers import StoppingCriteria, StoppingCriteriaList
import torch
import re

class SentenceEndCriteria(StoppingCriteria):
    def _init_(self, tokenizer):
        self.tokenizer = tokenizer

    def _call_(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        # Stop only if punctuation is NOT part of a numbered list
        if re.search(r"(?<!\d)[.?!](\s|\n)$", text):
            return True
        return False

def generate_response(prompt, max_new_tokens=800):
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,  # allow proper stop
            pad_token_id=tokenizer.eos_token_id
        )

    text = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    # 🛠 Ensure we don’t return a half-written last point
    lines = text.strip().split("\n")
    cleaned_lines = []
    for line in lines:
        if line.strip() and not line.strip().endswith((".", "!", "?")):
            break  # stop if a line ends without punctuation
        cleaned_lines.append(line)

    return "\n".join(cleaned_lines)




@st.cache_resource
def load_hf_sentiment():
    return pipeline("sentiment-analysis")

def analyze_sentiment(text):
    clf = load_hf_sentiment()
    res = clf(text)[0]
    return {"label": res["label"], "score": float(res["score"])}

tab1, tab2, tab3, tab4 = st.tabs(["🤖 Q&A", "📊 Budget Summary", "💡 Spending Insights", "🔍 NLU Analysis"])

with tab1:
    persona = st.selectbox("Persona", ["student", "professional", "general"])
    question = st.text_area("Your Question")
    if st.button("Get Advice"):
        st.write(generate_response(f"You are a helpful financial advisor for a {persona}. {question}"))

with tab2:
    income = st.number_input("Monthly Income", min_value=0.0, step=100.0)
    expenses = st.text_area("Expenses JSON", '{"rent": 1000, "food": 300}')
    goal = st.number_input("Savings Goal", min_value=0.0, step=50.0)
    if st.button("Analyze Budget"):
        ex = json.loads(expenses)
        total_exp = sum(ex.values())
        surplus = income - total_exp
        progress = (surplus / goal * 100) if goal else None
        st.json({"income": income, "total_expenses": total_exp, "surplus": surplus, "goal_progress_percent": progress})

with tab3:
    income2 = st.number_input("Monthly Income", min_value=0.0, step=100.0, key="si_income")
    expenses2 = st.text_area("Expenses JSON", '{"rent": 1000, "food": 300}', key="si_exp")
    goals2 = st.text_area("Goals JSON", '[{"name":"Laptop","target_amount":1200,"target_months":6}]')
    if st.button("Get Insights"):
        ex = json.loads(expenses2)
        goals_list = json.loads(goals2)
        surplus = income2 - sum(ex.values())
        insights = []
        for g in goals_list:
            monthly_needed = g["target_amount"] / g["target_months"]
            insights.append({"goal": g["name"], "achievable": surplus >= monthly_needed})
        st.json(insights)

with tab4:
    text_nlu = st.text_area("Enter text for sentiment analysis")
    if st.button("Analyze Sentiment"):
        st.json(analyze_sentiment(text_nlu))


Overwriting app.py


In [10]:
from pyngrok import ngrok
import threading, os, time

ngrok.kill()
PORT = 8501
public_url = ngrok.connect(PORT)
print("🌐 Streamlit UI:", public_url)

def run():
    os.system(f"streamlit run app.py --server.port {PORT}")

threading.Thread(target=run, daemon=True).start()
time.sleep(5)


🌐 Streamlit UI: NgrokTunnel: "https://a071784f8931.ngrok-free.app" -> "http://localhost:8501"
